In [ ]:
import pandas as pd
import numpy as np

unit_conversion = 1e9 # express results in nanoamp
nobias_path = "/home/sauron/NS_LOCAL/4-Misc/BatchData/BatchA/Resistance/PCBA9/BatchA_PCBA9_Dark_0V_2025-09-25_16-00-20/loop_001_FBA5CD_data.csv"
operating_path = "/home/sauron/NS_LOCAL/4-Misc/BatchData/BatchA/Resistance/PCBA9/BatchA_PCBA9_Dark_17p4V_2025-09-25_16-05-16/loop_005_70EA7F_data.csv"
nobias_data = pd.read_csv(nobias_path,header=None).to_numpy(dtype=float)
operating_data = pd.read_csv(operating_path,header=None).to_numpy(dtype=float)
# Mask out inactive pixels due to dead channels
boardbInactive = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,21,22,23,24]
mask = np.zeros((10,10), dtype=bool)
for i in boardbInactive:
    i-=1
    row = i//10
    col = i%10 
    mask[row,col] = True
#print(mask)
#print(np.sum(mask))
#print(nobias_data)
nobias_data = np.ma.masked_array(nobias_data,mask)
operating_data = np.ma.masked_array(operating_data,mask)
#print(nobias_data)

#div = np.divide(operating_data,nobias_data)
#threshold = 10 # dark current should rise at least 10 percent under bias. Probably
#nonresponsive = div<threshold
#num_nr = np.sum(nonresponsive)
#print(nonresponsive)
#print(div)

#operating_data = np.ma.masked_array(operating_data,nonresponsive)
#print(operating_data)
# Determine nonresponsive pixels using difference between 0V and operating point

dc_mean = np.mean(operating_data)*unit_conversion
dc_std = np.std(operating_data)*unit_conversion
#print(num_nr)
print(np.sum(mask))
print(np.ma.count(operating_data))
print(dc_mean)
print(dc_std)

#ratio = num_nr/(np.ma.count(operating_data) + num_nr)
#print(ratio)

19
81
1.1788035543209876
0.6694130371793879


# Resistance

In [27]:
import os
unit = 1e9 #gigaohm
resistance_dir = "/home/sauron/NS_LOCAL/4-Misc/BatchData/BatchA/Resistance/PCBA5/Resistance/"
I = []
for file in sorted(os.listdir(resistance_dir)):
    if file.endswith(".csv"):
        if "n" in file:
            data = pd.read_csv(resistance_dir+file,header=None).to_numpy(dtype=float)
            data *= -1
            I.append(data)
        else:
            I.append(pd.read_csv(resistance_dir+file,header=None).to_numpy(dtype=float))
        print(file)

def flag_open_pixels(R, z_thresh=4.0, min_R=1e-12):
    """
    R: array-like of resistances (2D or 1D). NaN/inf allowed.
    z_thresh: robust z-score threshold on log10(R).
    min_R: floor to avoid log10 underflow if you ever have tiny/zero entries.
    Returns: mask_open (bool array same shape as R), z (robust z-scores), info dict
    """
    R = np.asarray(R, dtype=float)
    mask_bad = ~np.isfinite(R) | (R <= 0)
    R_safe = np.where(mask_bad, np.nan, np.maximum(R, min_R))

    x = np.log10(R_safe)  # heavy-tail becomes manageable
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med))

    # Avoid division by ~0 if all values are similar
    if mad == 0 or not np.isfinite(mad):
        z = np.full_like(x, np.nan)
        mask_open = np.zeros_like(x, dtype=bool)
    else:
        z = 0.67448975 * (x - med) / mad  # 0.674... makes MAD comparable to std
        mask_open = (z > z_thresh)

    # Never flag already-bad entries as “open” by this logic; keep separate if you like
    mask_open = np.where(mask_bad, False, mask_open)

    info = {
        "log10_median": med,
        "log10_mad": mad,
        "z_thresh": z_thresh,
        "R_thresh_approx": 10 ** (med + (z_thresh / 0.67448975) * mad) if mad and np.isfinite(mad) else np.nan
    }
    return mask_open, z, info

voltages = [-5,-2.5,0,2.5,5]
voltages = np.array(voltages)
I = np.stack(I,axis=0)

vc = voltages-voltages.mean()
denom = (vc**2).sum()
slope = (vc[:, None, None] * (I - I.mean(axis=0, keepdims=True))).sum(axis=0) / denom  # dI/dV
with np.errstate(divide='ignore', invalid='ignore'):
    R = 1.0 / slope
R = np.ma.masked_array(R,mask)
#R = np.ma.masked_array(R,nonresponsive)


mask_open, z, info = flag_open_pixels(R, z_thresh=0.5)
print("Approx cutoff resistance:", info["R_thresh_approx"]/unit)
print("Open/disconnected pixel count:", mask_open.sum())
R = np.ma.masked_array(R,mask_open)
#print(R/unit)
print(np.mean(R)/unit)
print(np.std(R)/unit)
operating_data = np.ma.masked_array(operating_data,mask_open)
dc_mean = np.mean(operating_data)*unit_conversion
dc_std = np.std(operating_data)*unit_conversion
#print(num_nr)
#print(np.sum(mask))
#print(np.ma.count(operating_data))
print(dc_mean)
print(dc_std)

1_n5V.csv
2_n2p5V.csv
3_0V.csv
4_2p5V.csv
5_5V.csv
Approx cutoff resistance: 50.47563173024826
Open/disconnected pixel count: 38
20.0344580333796
10.095040278640676
1.2507308461538462
0.4226621894335911
